In [1]:
import subprocess as sp
import numpy as np
import pandas as pd
from io import StringIO
import os
import re
import shutil

In [2]:
#input and output dirs
datadirs = ["/global/cfs/cdirs/m1759/yswang/data/tf_fp32_amp_fw/"]
outputdir = "../results/tf_fp32_amp_fw"

# Functions

In [3]:
def transpose_frame(df_metrics):
    #Copy the profile frame to make sure not to overwrite it and potentially read it in again if we screwed it up
    selectkeys = ["ID", "Name"]
    resultkeys = ["Name"]
    
    tc_peak_perf_flops = 125*10**12
    
    #as metricdf use df_summary
    metricdf = df_metrics.copy()
    profiledf = pd.DataFrame(columns=selectkeys)

    ####### Get timing information
    ### CUDA Time
    # get cycles
    metricname = "CUDA Cycles"
    cyclesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__cycles_elapsed") & (metricdf["Metric Type"]=="total"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # get rates
    metricname = "CUDA Rates"
    ratesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__cycles_elapsed") & (metricdf["Metric Type"]=="rate"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # check consistency
    if not cyclesdf[['ID', 'Name']].equals(ratesdf[['ID', 'Name']]):
        raise ValueError("CUDA Time data not consistent")
    # adjust metric unit
    ratesdf.loc[ratesdf["Metric Unit"].str.contains("cycle/nsecond"), ["CUDA Rates"]] *= 1e9
    # manual merge and compute CUDA Time
    cyclesdf["CUDA Rates"] = list(ratesdf["CUDA Rates"])
    cyclesdf["CUDA Time"] = cyclesdf["CUDA Cycles"] / cyclesdf["CUDA Rates"]
    # merge with output
    profiledf = cyclesdf[selectkeys+['CUDA Time']].copy()
    
    ### Tensor Core Time
    # get cycles
    metricname = "TC Cycles"
    cyclesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__pipe_tensor_op_hmma_cycles_active") & (metricdf["Metric Type"]=="total"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # get rates
    metricname = "TC Rates"
    ratesdf = metricdf.loc[(metricdf["Metric Name"]=="smsp__pipe_tensor_op_hmma_cycles_active") & (metricdf["Metric Type"]=="rate"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # check consistency
    if not cyclesdf[['ID', 'Name']].equals(ratesdf[['ID', 'Name']]):
        raise ValueError("TC Time data not consistent")
    # adjust metric unit
    ratesdf.loc[ratesdf["Metric Unit"].str.contains("cycle/nsecond"), ["TC Rates"]] *= 1e9
    # manual merge and compute CUDA Time
    cyclesdf["TC Rates"] = list(ratesdf["TC Rates"])
    cyclesdf["TC Time"] = cyclesdf["TC Cycles"] / cyclesdf["TC Rates"]
    # merge & cleanup
    profiledf = profiledf.merge(cyclesdf[selectkeys+['TC Time']], on=selectkeys, how="outer").fillna(0.)
    
    ### Combine
    del profiledf['ID']
    del metricdf['ID']
    profiledf['Invocations'] = 1
    profiledf = profiledf.groupby(resultkeys).sum().reset_index()
    #profiledf.sort_values(by=resultkeys, inplace=True)
    #profiledf.reset_index(drop=True, inplace=True)

    ####### Get number of FLOPs
    
    ### FMA FLOPs = number of FMA instructions x 2
    metricdf.loc[metricdf["Metric Name"].str.contains("fma"), ["Metric Value"]] *= 2
    
    ### FP32 FLOPs
    metrics = ['smsp__sass_thread_inst_executed_op_fadd_pred_on',
               'smsp__sass_thread_inst_executed_op_ffma_pred_on',
               'smsp__sass_thread_inst_executed_op_fmul_pred_on']
    tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "FP32 FLOPs"})
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["FP32 FLOPs"]], on=resultkeys, how="inner")
    
    ### FP16 FLOPs
    metrics = ['smsp__sass_thread_inst_executed_op_hadd_pred_on',
               'smsp__sass_thread_inst_executed_op_hfma_pred_on',
               'smsp__sass_thread_inst_executed_op_hmul_pred_on']
    tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "FP16 FLOPs"})
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["FP16 FLOPs"]], on=resultkeys, how="inner")
    
    #### TC FLOPs
    tmpdf = metricdf.loc[ metricdf["Metric Name"].str.contains("sm__inst_executed_pipe_tensor_op_hmma"), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "TC Utilization"})
    tmpdf["TC Utilization"] = 0.01 * tmpdf["TC Utilization"]
    profiledf = profiledf.merge(tmpdf[resultkeys+["TC Utilization"]], on=resultkeys, how="inner")
    profiledf["TC Utilization"] = profiledf["TC Utilization"] / profiledf['Invocations']
    profiledf["TC FLOPs"] = tc_peak_perf_flops * profiledf["TC Utilization"] * profiledf["TC Time"]
    
    ### Total FLOPs
    profiledf["FLOPs"] = profiledf["FP32 FLOPs"] + profiledf["FP16 FLOPs"] + profiledf["TC FLOPs"] #+ metricdf["FP64 FLOPs"]
    
    ### FLOPs fractions
    #profiledf["FP64 FLOPs Fraction"] = profiledf["FP64 FLOPs"]/profiledf["FLOPs"]
    profiledf["FP32 FLOPs Fraction"] = profiledf["FP32 FLOPs"]/profiledf["FLOPs"]
    profiledf["FP16 FLOPs Fraction"] = profiledf["FP16 FLOPs"]/profiledf["FLOPs"]
    profiledf["TC FLOPs Fraction"]   = profiledf["TC FLOPs"]/profiledf["FLOPs"]
    
    ####### Get number of bytes
    
    ### Shared transactions
    #project out
    shareddf = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__data_pipe_lsu_wavefronts_mem_shared_op"), resultkeys+["Metric Value"] ].copy()
    shareddf = shareddf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "Shared Transactions"})
    #add to timings
    profiledf = profiledf.merge(shareddf[resultkeys+["Shared Transactions"]], on=resultkeys, how="inner")
    
    ### L1 atomic transactions
    # project out
    metrics = ['l1tex__t_set_accesses_pipe_lsu_mem_global_op_atom',
               'l1tex__t_set_accesses_pipe_lsu_mem_global_op_red',
               'l1tex__t_set_accesses_pipe_tex_mem_surface_op_atom',
               'l1tex__t_set_accesses_pipe_tex_mem_surface_op_red']
    atomicdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    # get reads and writes
    atomicdf = atomicdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "L1 Atomic Transactions"})
    # add to timings
    profiledf = profiledf.merge(atomicdf[resultkeys+["L1 Atomic Transactions"]], on=resultkeys, how="inner")

    ### Local transactions 
    # project out
    localdf = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__t_sectors_pipe_lsu_mem_local_op"), resultkeys+["Metric Value"] ].copy()
    localdf = localdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "Local Transactions"})
    # add to timings
    profiledf = profiledf.merge(localdf[resultkeys+["Local Transactions"]], on=resultkeys, how="inner")
    
    ### Global transactions 
    # project out
    globaldf = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__t_sectors_pipe_lsu_mem_global_op"), resultkeys+["Metric Value"] ].copy()
    globaldf = globaldf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "Global Transactions"})
    # add to timings
    profiledf = profiledf.merge(globaldf[resultkeys+["Global Transactions"]], on=resultkeys, how="inner")
    
    ### L1 Bytes
    profiledf["L1 Transactions"] = (profiledf["Shared Transactions"] + profiledf["L1 Atomic Transactions"]
                            + profiledf["Local Transactions"] + profiledf["Global Transactions"])
    profiledf["L1 Bytes"] = profiledf["L1 Transactions"] * 32
    
    ### L2 atomic & reduction
    metricdf.loc[(metricdf["Metric Name"].str.contains("lts__t_sectors_op")) & (metricdf["Metric Type"]=="total"), ["Metric Value"]] *= 2

    ### L2 transactions
    # project out
    l2df = metricdf.loc[metricdf["Metric Name"].str.contains("lts__t_sectors_op"), resultkeys+["Metric Value"] ].copy()
    l2df = l2df.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "L2 Transactions"})
    l2df["L2 Bytes"] = l2df["L2 Transactions"] * 32
    # add to timings
    profiledf = profiledf.merge(l2df[resultkeys+["L2 Transactions", "L2 Bytes"]], on=resultkeys, how="inner")

    ### DRAM Bytes
    # project out
    dramdf = metricdf.loc[metricdf["Metric Name"].str.contains("dram__sectors"), resultkeys+["Metric Value"] ].copy()
    dramdf = dramdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "DRAM Transactions"})
    dramdf["DRAM Bytes"] = dramdf["DRAM Transactions"] * 32
    # add to timings
    profiledf = profiledf.merge(dramdf[resultkeys+["DRAM Transactions", "DRAM Bytes"]], on=resultkeys, how="inner")
        
    ### Host Memory Bytes
    # project out
    sysmemdf = metricdf.loc[metricdf["Metric Name"].str.contains("lts__t_sectors_aperture_sysmem_op"), resultkeys+["Metric Value"] ].copy()
    sysmemdf = sysmemdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "SYSMEM Transactions"})
    sysmemdf["SYSMEM Bytes"] = sysmemdf["SYSMEM Transactions"] * 32
    # add to timings
    profiledf = profiledf.merge(sysmemdf[resultkeys+["SYSMEM Transactions", "SYSMEM Bytes"]], on=resultkeys, how="inner")
    
    
    ### Get performance
    profiledf["Performance GFlop/s"]      = profiledf["FLOPs"]      / (profiledf["CUDA Time"]*10**9)
    profiledf["FP32 Performance GFlop/s"] = profiledf["FP32 FLOPs"] / (profiledf["CUDA Time"]*10**9)
    profiledf["FP16 Performance GFlop/s"] = profiledf["FP16 FLOPs"] / (profiledf["CUDA Time"]*10**9)
    profiledf["TC Performance GFlop/s"]   = profiledf["TC FLOPs"]   / (profiledf["TC Time"]*10**9)


    ### Get AI
    # L1
    profiledf["L1 AI"]        = profiledf["FLOPs"]      / profiledf["L1 Bytes"]
    profiledf["FP32 L1 AI"]   = profiledf["FP32 FLOPs"] / profiledf["L1 Bytes"]
    profiledf["FP16 L1 AI"]   = profiledf["FP16 FLOPs"] / profiledf["L1 Bytes"]
    profiledf["TC L1 AI"]     = profiledf["TC FLOPs"]   / profiledf["L1 Bytes"]
    # L2
    profiledf["L2 AI"]        = profiledf["FLOPs"]      / profiledf["L2 Bytes"]
    profiledf["FP32 L2 AI"]   = profiledf["FP32 FLOPs"] / profiledf["L2 Bytes"]
    profiledf["FP16 L2 AI"]   = profiledf["FP16 FLOPs"] / profiledf["L2 Bytes"]
    profiledf["TC L2 AI"]     = profiledf["TC FLOPs"]   / profiledf["L2 Bytes"]
    # DRAM
    profiledf["DRAM AI"]      = profiledf["FLOPs"]      / profiledf["DRAM Bytes"]
    profiledf["FP32 DRAM AI"] = profiledf["FP32 FLOPs"] / profiledf["DRAM Bytes"]
    profiledf["FP16 DRAM AI"] = profiledf["FP16 FLOPs"] / profiledf["DRAM Bytes"]
    profiledf["TC DRAM AI"]   = profiledf["TC FLOPs"]   / profiledf["DRAM Bytes"]
    # SYSMEM
    profiledf["SYSMEM AI"]      = profiledf["FLOPs"]      / profiledf["SYSMEM Bytes"]
    profiledf["FP32 SYSMEM AI"] = profiledf["FP32 FLOPs"] / profiledf["SYSMEM Bytes"]
    profiledf["FP16 SYSMEM AI"] = profiledf["FP16 FLOPs"] / profiledf["SYSMEM Bytes"]
    profiledf["TC SYSMEM AI"]   = profiledf["TC FLOPs"]   / profiledf["SYSMEM Bytes"]

    
    ### Cleanup
    profiledf.sort_values(by=resultkeys).reset_index(drop=True, inplace=True)
    #print(metricdf[['CUDA Time Avg', 'TC Time Avg']])
    
    return profiledf

# Import Data

In [4]:
#get all the files
files = []
for datadir in datadirs:
    files += [ os.path.join(datadir,x) for x in os.listdir(datadir) if ((os.path.splitext(x)[-1] == ".csv"))]

#recs
records = []

#build feature list:
for path in files:
    
    #filename
    file = os.path.basename(path)
    
    #path
    path = os.path.dirname(path)
    
    #splitup
    splt = file.split(".")
    
    prefix = ".".join(splt[0:-1])
    
    #append to records
    records.append({"prefix": prefix, "file": os.path.join(path, file)})

#put in df
recorddf = pd.DataFrame(records).sort_values(["prefix"]).reset_index(drop=True)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
display(recorddf)
tmp = list(recorddf['file'])
print(tmp)

,prefix,file
0,full_profile,/global/cfs/cdirs/m1759/yswang/data/tf_fp32_am...
1,full_profile_ssd,/global/cfs/cdirs/m1759/yswang/data/tf_fp32_am...


['/global/cfs/cdirs/m1759/yswang/data/tf_fp32_amp_fw/full_profile.csv', '/global/cfs/cdirs/m1759/yswang/data/tf_fp32_amp_fw/full_profile_ssd.csv']


In [5]:
#metrics
#for fin in recorddf["file"]:
for fin in ['/global/cfs/cdirs/m1759/yswang/data/tf_fp32_amp_fw/full_profile_ssd.csv']:
    #project frame
    print("\n### Processing "+fin)
    metricdf = pd.read_csv(fin)
    
    #fuse read/write metrics together:
    unique_metrics = metricdf["Metric Name"].unique()
    unique_metrics = set([x.replace(".sum","").replace("_write","").replace("_read","").replace("_ld","").replace("_st","") for x in unique_metrics])
    #add the metric type
    metricdf["Metric Type"] = "total"
    #read
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_read"), "Metric Type" ] = "read"
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_ld"), "Metric Type" ] = "read"
    #write
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_write"), "Metric Type" ] = "write"
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains("_st"), "Metric Type" ] = "write"
    #rate
    metricdf.loc[ metricdf[ "Metric Name" ].str.contains(".per_second"), "Metric Type" ] = "rate"

    for metric in unique_metrics:
        metricdf.loc[ metricdf[ "Metric Name"].str.startswith(metric), "Metric Name" ] = metric
    #cleanups
    tmpdf = metricdf[['ID', "Kernel Name", "Metric Name", "Metric Unit", "Metric Type", "Metric Value"]].rename(columns={"Kernel Name": 'Name'})

    #compute the profile
    profiledf = transpose_frame(tmpdf)
    #print(profiledf)


### Processing /global/cfs/cdirs/m1759/yswang/data/tf_fp32_amp_fw/full_profile_ssd.csv


In [6]:
#profiledf = profiledf.groupby('Name').sum().reset_index()
profiledf = profiledf.sort_values('CUDA Time', ascending=False).reset_index(drop=True)
display(profiledf)
#print(profiledf['CUDA Time'].sum())
#print(profiledf['TC Time'].sum())

,Name,CUDA Time,TC Time,Invocations,FP32 FLOPs,FP16 FLOPs,TC Utilization,TC FLOPs,FLOPs,FP32 FLOPs Fraction,...,FP16 L2 AI,TC L2 AI,DRAM AI,FP32 DRAM AI,FP16 DRAM AI,TC DRAM AI,SYSMEM AI,FP32 SYSMEM AI,FP16 SYSMEM AI,TC SYSMEM AI
0,Volta_hmma_implicit_gemm_fprop_fp32_nhwc_128x1...,0.065602,0.065602,8,1.953497e+09,0.000000e+00,0.742000,6.084577e+12,6.086531e+12,0.000321,...,0.000000,93.784633,978.214529,0.313962,0.000000,977.900567,3.396502e+09,1.090121e+06,0.000000,3.395411e+09
1,void tensorflow::BiasNHWCKernel<Eigen::half>(i...,0.028810,0.000000,29,0.000000e+00,2.461336e+09,0.000000,0.000000e+00,2.461336e+09,0.000000,...,0.249928,0.000000,0.249358,0.000000,0.249358,0.000000,3.789002e+05,0.000000e+00,378900.177340,0.000000e+00
2,volta_fp16_s884cudnn_fp16_256x128_ldg8_relu_f2...,0.018991,0.018991,12,2.141716e+08,1.338573e+07,0.784108,1.861347e+12,1.861574e+12,0.000115,...,0.001293,179.824302,789.834616,0.090869,0.005679,789.738067,6.925499e+08,7.967695e+04,4979.809524,6.924653e+08
3,volta_fp16_s884cudnn_fp16_128x128_ldg8_dgrad_f...,0.014707,0.014707,3,1.196425e+09,0.000000e+00,0.784133,1.441497e+12,1.442694e+12,0.000829,...,0.000000,74.015077,377.019521,0.312662,0.000000,376.706859,2.146866e+09,1.780395e+06,0.000000,2.145085e+09
4,void tensorflow::functor::PadInputCustomKernel...,0.009645,0.000000,3,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
5,volta_fp16_s884gemm_fp16_128x128_ldg8_f2f_nn,0.008899,0.008899,36,7.219446e+08,0.000000e+00,0.582933,6.484711e+11,6.491930e+11,0.001112,...,0.000000,62.032444,252.106406,0.280359,0.000000,251.826047,8.050509e+07,8.952686e+04,0.000000,8.041556e+07
6,"void tensorflow::S2B<Eigen::half,int=2,bool=0>...",0.008450,0.000000,12,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
7,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.007567,0.000000,9,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
8,"void foldedNhwcToNhwcKernel<__half,__half,floa...",0.006319,0.000000,3,5.967560e+08,5.962644e+08,0.000000,0.000000e+00,1.193020e+09,0.500206,...,0.249997,0.000000,0.499518,0.249862,0.249656,0.000000,1.775328e+06,8.880297e+05,887298.285714,0.000000e+00
9,volta_fp16_s884cudnn_fp16_128x128_ldg8_relu_f2...,0.003464,0.003464,1,4.529848e+08,2.831155e+07,0.710800,3.077878e+11,3.082691e+11,0.001469,...,0.006368,69.228259,446.154633,0.655600,0.040975,445.458058,1.376201e+09,2.022254e+06,126390.857143,1.374053e+09


# Compute AI Results

In [7]:
#sum over all kernels
combinedselectkeys = ["Precision", "Network Name", "Batch Size", "Pass"]

#copy profiledf
combineddf = profiledf.copy()
combineddf['Precision'] = 'mixed'
combineddf['Network Name'] = 'deepCam'
combineddf['Batch Size'] = 2
combineddf['Pass'] = 'forward'

#sum up
combineddf = combineddf.groupby(by=combinedselectkeys).sum()#.reset_index()


#the flop fractions need to be recomputed
combineddf["FP32 FLOPs Fraction"] = combineddf["FP32 FLOPs"] / combineddf["FLOPs"]
combineddf["FP16 FLOPs Fraction"] = combineddf["FP16 FLOPs"] / combineddf["FLOPs"]
combineddf["TC FLOPs Fraction"]   = combineddf["TC FLOPs"]   / combineddf["FLOPs"]

### Get performance
combineddf["Performance GFlop/s"]      = combineddf["FLOPs"]      / (combineddf["CUDA Time"]*10**9)
combineddf["FP32 Performance GFlop/s"] = combineddf["FP32 FLOPs"] / (combineddf["CUDA Time"]*10**9)
combineddf["FP16 Performance GFlop/s"] = combineddf["FP16 FLOPs"] / (combineddf["CUDA Time"]*10**9)
combineddf["TC Performance GFlop/s"]   = combineddf["TC FLOPs"]   / (combineddf["TC Time"]*10**9)


### Get AI
# L1
combineddf["L1 AI"]        = combineddf["FLOPs"]      / combineddf["L1 Bytes"]
combineddf["FP32 L1 AI"]   = combineddf["FP32 FLOPs"] / combineddf["L1 Bytes"]
combineddf["FP16 L1 AI"]   = combineddf["FP16 FLOPs"] / combineddf["L1 Bytes"]
combineddf["TC L1 AI"]     = combineddf["TC FLOPs"]   / combineddf["L1 Bytes"]
# L2
combineddf["L2 AI"]        = combineddf["FLOPs"]      / combineddf["L2 Bytes"]
combineddf["FP32 L2 AI"]   = combineddf["FP32 FLOPs"] / combineddf["L2 Bytes"]
combineddf["FP16 L2 AI"]   = combineddf["FP16 FLOPs"] / combineddf["L2 Bytes"]
combineddf["TC L2 AI"]     = combineddf["TC FLOPs"]   / combineddf["L2 Bytes"]
# DRAM
combineddf["DRAM AI"]      = combineddf["FLOPs"]      / combineddf["DRAM Bytes"]
combineddf["FP32 DRAM AI"] = combineddf["FP32 FLOPs"] / combineddf["DRAM Bytes"]
combineddf["FP16 DRAM AI"] = combineddf["FP16 FLOPs"] / combineddf["DRAM Bytes"]
combineddf["TC DRAM AI"]   = combineddf["TC FLOPs"]   / combineddf["DRAM Bytes"]

combineddf.sort_values(by=combinedselectkeys).reset_index(drop=True, inplace=True)

In [8]:
display(combineddf)

,,,,CUDA Time,TC Time,Invocations,FP32 FLOPs,FP16 FLOPs,TC Utilization,TC FLOPs,FLOPs,FP32 FLOPs Fraction,FP16 FLOPs Fraction,...,FP16 L2 AI,TC L2 AI,DRAM AI,FP32 DRAM AI,FP16 DRAM AI,TC DRAM AI,SYSMEM AI,FP32 SYSMEM AI,FP16 SYSMEM AI,TC SYSMEM AI
Precision,Network Name,Batch Size,Pass,,,,,,,,,,,,,,,,,,,,,
mixed,deepCam,2,forward,0.197169,0.11616,525,3.494666e+10,4.194180e+09,6.296758,1.072454e+13,1.076368e+13,0.003247,0.00039,...,0.029046,74.271713,231.336538,0.751085,0.090143,230.49531,8.709467e+09,1.387262e+08,2.216420e+06,8.568525e+09


# Export Data

In [9]:
profiledf.to_csv(os.path.join(outputdir,"full_profile.csv"), encoding='utf-8', index=False)
combineddf.to_csv(os.path.join(outputdir,"combined_profile.csv"), encoding='utf-8', index=False)